<a href="https://colab.research.google.com/github/Ziaad03/ngram-language-model-sentiment-classification/blob/main/NaiveBayesClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

from datasets import load_dataset
import numpy as np
from collections import Counter

# This downloads the dataset and loads its splits (e.g., train, validation, test)
dataset = load_dataset("stanfordnlp/sst")

# Print details about the dataset splits
print(dataset)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.68k [00:00<?, ?B/s]

sst.py:   0%|          | 0.00/9.13k [00:00<?, ?B/s]

The repository for stanfordnlp/sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/stanfordnlp/sst.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})


In [3]:
# Print train
dataset['train']

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 8544
})

In [4]:
print("Columns:", dataset["train"].column_names)
print("Features:", dataset["train"].features)


Columns: ['sentence', 'label', 'tokens', 'tree']
Features: {'sentence': Value(dtype='string', id=None), 'label': Value(dtype='float32', id=None), 'tokens': Value(dtype='string', id=None), 'tree': Value(dtype='string', id=None)}


In [5]:
print(dataset["train"][0])


{'sentence': "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .", 'label': 0.6944400072097778, 'tokens': "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.", 'tree': '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'}


In [6]:
df = dataset["train"].to_pandas()
print(df.head())


                                            sentence    label  \
0  The Rock is destined to be the 21st Century 's...  0.69444   
1  The gorgeously elaborate continuation of `` Th...  0.83333   
2  Singer\/composer Bryan Adams contributes a sle...  0.62500   
3  You 'd think by now America would have had eno...  0.50000   
4               Yet the act is still charming here .  0.72222   

                                              tokens  \
0  The|Rock|is|destined|to|be|the|21st|Century|'s...   
1  The|gorgeously|elaborate|continuation|of|``|Th...   
2  Singer\/composer|Bryan|Adams|contributes|a|sle...   
3  You|'d|think|by|now|America|would|have|had|eno...   
4               Yet|the|act|is|still|charming|here|.   

                                                tree  
0  70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|5...  
1  71|70|69|69|67|67|66|64|63|62|62|61|61|58|57|5...  
2  72|71|71|70|68|68|67|67|66|63|62|62|60|60|58|5...  
3  36|35|34|33|33|32|30|29|27|26|25|24|23|23|22|2...

In [7]:
print("Number of training samples:", len(dataset["train"]))
print("Number of validation samples:", len(dataset["validation"]))
print("Number of test samples:", len(dataset["test"]))

Number of training samples: 8544
Number of validation samples: 1101
Number of test samples: 2210


In [8]:
#extract the data
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

#print
print("Train data:", train_data[0])
print("Validation data:", validation_data[0])
print("Test data:", test_data[0])

Train data: {'sentence': "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .", 'label': 0.6944400072097778, 'tokens': "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.", 'tree': '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'}
Validation data: {'sentence': "It 's a lovely film with lovely performances by Buy and Accorsi .", 'label': 0.791670024394989, 'tokens': "It|'s|a|lovely|film|with|lovely|performances|by|Buy|and|Accorsi|.", 'tree': '25|23|21|20|20|19|17|17|16|14|14|15|24|15|16|18|18|19|22|21|22|23|24|25|0'}
Test data: {'sentence': 'Effective but too-tepid bio

# **Preprocessing for Classification**

In [9]:

print(train_data)

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 8544
})


In [10]:
def map_labels(data):
    label = data["label"]
    if 0 <= label <= 0.2:
        data["label"] = 0
    elif 0.2 < label <= 0.4:
        data["label"] = 1
    elif 0.4 < label <= 0.6:
        data["label"] = 2
    elif 0.6 < label <= 0.8:
        data["label"] = 3
    elif 0.8 < label <= 1:
        data["label"] = 4
    return data

train_data = train_data.map(map_labels)
validation_data =validation_data.map(map_labels)
test_data = test_data.map(map_labels)

# Check a few records
df = train_data.to_pandas()
print(df.head())

df = validation_data.to_pandas()
print(df.head())

df = test_data.to_pandas()
print(df.head())


Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

                                            sentence  label  \
0  The Rock is destined to be the 21st Century 's...    3.0   
1  The gorgeously elaborate continuation of `` Th...    4.0   
2  Singer\/composer Bryan Adams contributes a sle...    3.0   
3  You 'd think by now America would have had eno...    2.0   
4               Yet the act is still charming here .    3.0   

                                              tokens  \
0  The|Rock|is|destined|to|be|the|21st|Century|'s...   
1  The|gorgeously|elaborate|continuation|of|``|Th...   
2  Singer\/composer|Bryan|Adams|contributes|a|sle...   
3  You|'d|think|by|now|America|would|have|had|eno...   
4               Yet|the|act|is|still|charming|here|.   

                                                tree  
0  70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|5...  
1  71|70|69|69|67|67|66|64|63|62|62|61|61|58|57|5...  
2  72|71|71|70|68|68|67|67|66|63|62|62|60|60|58|5...  
3  36|35|34|33|33|32|30|29|27|26|25|24|23|23|22|2...  
4        

In [11]:
filtered_dataset = train_data.filter(lambda example: example["label"] == 2)
docsClassNumber = filtered_dataset
print(docsClassNumber)


Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 1624
})


In [12]:
#get the labels
train_labels = train_data["label"]
validation_labels = validation_data["label"]
test_labels = test_data["label"]


print("Train labels:", train_labels)
print("Validation labels:", validation_labels)
print("Test labels:", test_labels)

Train labels: [3.0, 4.0, 3.0, 2.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 2.0, 4.0, 2.0, 4.0, 3.0, 3.0, 2.0, 4.0, 3.0, 4.0, 3.0, 2.0, 4.0, 2.0, 2.0, 4.0, 3.0, 4.0, 1.0, 4.0, 3.0, 2.0, 3.0, 4.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 4.0, 2.0, 3.0, 2.0, 0.0, 3.0, 3.0, 1.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 1.0, 1.0, 4.0, 3.0, 4.0, 4.0, 3.0, 2.0, 1.0, 3.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 2.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 1.0, 4.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 3.0, 3.0, 3.0, 4.0, 1.0, 2.0, 2.0, 2.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 2.0, 4.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 2.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 4.0, 3.0, 1.0, 

# **Naive Bayes Classification**

In [25]:
def get_vocabulary(data):

    vocabs = set()

    for token_sequence in data['tokens']:
        word_chars = []
        for token in token_sequence:
            if token == "|":
                # Join the characters into a full word.
                if word_chars:
                    word_str = ''.join(word_chars)
                    # Append if the word is longer than one character.
                    if len(word_str) > 1:
                        vocabs.add(word_str)
                    word_chars = []
            else:
                # Only append if the token is alphabetic or numeric.
                if token.isalpha() or token.isdigit():
                    word_chars.append(token)
        # In case the sequence doesn't end with a delimiter.
        if word_chars:
            word_str = ''.join(word_chars)
            if len(word_str) > 1:
                vocabs.add(word_str)

    return vocabs





In [26]:
print(get_vocabulary(train_data))
print(len(get_vocabulary(train_data)))

{'madcap', 'Frankie', 'achieving', 'Zone', 'twofifths', 'push', 'Neuwirth', 'corn', 'Julianne', 'servants', 'river', 'humanly', 'associate', 'crashing', 'AmericanRussian', 'facetoface', 'dollars', 'bask', 'zingers', 'provoked', 'freezers', 'unhurried', 'difficulties', 'safe', 'feeds', 'graduation', 'suffer', 'McWilliams', 'motifs', 'Leaks', 'Tara', 'breakthrough', 'yields', 'delusions', 'revision', 'suicidal', 'Dey', 'constantly', 'whimsical', 'Davies', 'captivating', 'except', 'Maid', 'copflick', 'transitions', 'Turturro', 'hilariously', 'borders', 'hang', 'exuberant', 'ghosts', 'Prepare', 'gripping', 'Of', 'elbows', 'duking', 'adventurous', 'spirituality', 'president', 'swimfan', 'commenting', 'Bai', 'vacuous', 'annoying', 'parties', 'blond', 'flirts', 'lives', 'starstudded', 'embraced', 'recognize', 'merely', 'Crossroads', 'eventual', 'morally', 'Fiji', 'roads', 'stench', 'holy', 'dramatics', 'bust', 'commitment', 'backlash', 'steaming', 'dominant', 'percussion', 'decades', 'obvious

In [24]:

# def get_vocabulary(train_data):
#     """
#     Extract unique words from the 'tokens' field of the dataset.

#     Args:
#         train_data (Dataset): The 'train' split.

#     Returns:
#         set: Unique words (lowercased) from all documents.
#     """
#     vocab = set()
#     for doc in train_data:
#         # Split 'tokens' into words and lowercase them
#         tokens = doc['tokens'].split('|')
#         for token in tokens:
#             if token:  # Skip empty tokens
#                 vocab.add(token.lower())
#     return sorted(vocab)

# print(get_vocabulary(train_data))
# print(len(get_vocabulary(train_data)))

['!', '!?', '#', '$', '&', "'", "''", "'30s", "'40s", "'50s", "'53", "'60s", "'70s", "'80s", "'90s", "'d", "'em", "'ll", "'m", "'n", "'n'", "'re", "'s", "'til", "'ve", '(', ')', '+', ',', '-', '--', '.', '...', '1', '1.2', '1.8', '10', '10,000', '10-course', '10-year', '10-year-old', '100', '100-minute', '100-year', '101', '102-minute', '103-minute', '104', '105', '10th', '10th-grade', '11', '110', '112-minute', '12', '12-year-old', '120', '127', '129-minute', '12th', '13', '13th', '14-year-old', '140', '146', '15', '15-year', '15th', '163', '168-minute', '170', '1790', '18', '18-year-old', '1899', '19', '1915', '1920', '1930s', '1933', '1937', '1938', '1940s', '1950', '1950s', '1952', '1953', '1957', '1958', '1959', '1960', '1960s', '1962', '1970', '1970s', '1971', '1972', '1973', '1975', '1979', '1980', '1980s', '1984', '1986', '1987', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1997', '1998', '1999', '19th', '19th-century', '1\\/2', '2', '2,500', '2-day', '20', '20,000'

In [18]:
def getWordOccurenceInClassDocs(word, classDocs):

    total_count = 0
    word = word.lower()  # Convert word to lowercase for case-insensitive matching

    for doc in classDocs:
        tokens_str = doc['tokens']  # Get the 'tokens' string (e.g., "The|Rock|is|destined")
        words = tokens_str.lower().split('|')  # Split into list of words, lowercase
        total_count += words.count(word)  # Add count of word in this document

    return total_count

In [29]:
def naive_bayes_classifier(train_data, classes):

    # Total number of documents
    docs_number = len(train_data)

    # Get the vocabulary (unique words) from the 'tokens' field
    vocab = get_vocabulary(train_data)

    # Initialize dictionaries for probabilities
    log_prior = {}
    loglikelihood = {}

    # Process each class
    for c in classes:
        # Filter documents belonging to class c
        class_docs = train_data.filter(lambda example: example['label'] == c)
        docs_class_number = len(class_docs)

        # Compute log prior: log(P(c)) = log(#docs in c / total #docs)
        log_prior[c] = np.log(docs_class_number / docs_number) if docs_class_number > 0 else -np.inf

        # Count word frequencies in this class
        word_freq = Counter()
        for doc in class_docs:
            # Split 'tokens' into words and convert to lowercase for consistency
            words = doc['tokens'].lower().split('|')
            word_freq.update(words)

        # Total number of word occurrences in this class
        total_word_occurrences = sum(word_freq.values())

        # Compute log likelihood for each word in the vocabulary
        loglikelihood[c] = {}
        for word in vocab:
            # Get word count in this class (case-insensitive)
            count = word_freq.get(word.lower(), 0)
            # Compute P(word|c) with add-one (Laplace) smoothing and take log
            loglikelihood[c][word] = np.log((count + 1) / (total_word_occurrences + len(vocab)))

    return log_prior, loglikelihood, vocab

# **Train the classfier**

In [30]:
# Train
log_prior, loglikelihood, vocab = naive_bayes_classifier(train_data, [0, 1, 2, 3, 4])
print(log_prior)
print(loglikelihood)
print(vocab)

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

{0: -2.0572184048951256, 1: -1.3486233932896636, 2: -1.6603370404783528, 3: -1.3028003989421393, 4: -1.892138654535677}
{0: {'madcap': -9.87942536618537, 'Frankie': -10.572572546745315, 'achieving': -10.572572546745315, 'Zone': -9.87942536618537, 'twofifths': -10.572572546745315, 'push': -9.87942536618537, 'Neuwirth': -10.572572546745315, 'corn': -9.87942536618537, 'Julianne': -10.572572546745315, 'servants': -10.572572546745315, 'river': -10.572572546745315, 'humanly': -10.572572546745315, 'associate': -10.572572546745315, 'crashing': -9.87942536618537, 'AmericanRussian': -10.572572546745315, 'facetoface': -10.572572546745315, 'dollars': -10.572572546745315, 'bask': -10.572572546745315, 'zingers': -10.572572546745315, 'provoked': -9.87942536618537, 'freezers': -10.572572546745315, 'unhurried': -10.572572546745315, 'difficulties': -10.572572546745315, 'safe': -10.572572546745315, 'feeds': -10.572572546745315, 'graduation': -10.572572546745315, 'suffer': -9.87942536618537, 'McWilliams':

# **Test the classfier**

In [36]:
def test_naive_bayes(testdoc, logprior, loglikelihood, C, V):

    sum_scores = {}

    #  iterate over each class
    for c in C:
        sum_scores[c] = logprior[c]  # Start with the log prior for this class
        #  process each word in the document for this class
        for word in testdoc:
            if word in V:
                sum_scores[c] += loglikelihood[c][word]  # Add log likelihood

    # Predict the class with the highest score
    predicted_class = max(sum_scores, key=sum_scores.get)
    return predicted_class

In [40]:
print(test_data[0]['tokens'].split('|'))
predicted_class = test_naive_bayes(test_data[0]['tokens'].split('|'), log_prior, loglikelihood, [0, 1, 2, 3, 4], vocab)
print(predicted_class)

['Effective', 'but', 'too-tepid', 'biopic']
3


# **Comparison with scikit-learn**

In [43]:
test_tokens = [doc["tokens"] for doc in test_data]
train_tokens = [doc["tokens"] for doc in train_data]
# Convert tokens to space-separated strings
test_texts = [" ".join(tokens.split("|")) for tokens in test_tokens]
train_texts = [" ".join(tokens.split("|")) for tokens in train_tokens]

print(test_texts[0])

Effective but too-tepid biopic


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

vectorizer = CountVectorizer(lowercase=True, token_pattern=r"\b\w+\b")
X_train = vectorizer.fit_transform(train_texts)
x_test = vectorizer.transform(test_texts)
# X_test is now a sparse matrix of word counts

from sklearn.naive_bayes import MultinomialNB

# Train
clf = MultinomialNB(alpha=1.0)  # Laplace smoothing
clf.fit(X_train, train_labels)

# Predict on the same data
predictions = clf.predict(x_test)

accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions, average='weighted')
recall = recall_score(test_labels, predictions, average='weighted')
f1 = f1_score(test_labels, predictions, average='weighted')

# Print results for comparison
print(f"Scikit-learn Accuracy: {accuracy}")
print(f"Scikit-learn Precision: {precision}")
print(f"Scikit-learn Recall: {recall}")
print(f"Scikit-learn F1-Score: {f1}")

Scikit-learn Accuracy: 0.40452488687782806
Scikit-learn Precision: 0.40972715492701367
Scikit-learn Recall: 0.40452488687782806
Scikit-learn F1-Score: 0.3565956313574668
